In [1]:
import json
from collections import Counter
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
import utils.molecule_tools as mol

/home/tweaving/.local/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


In [5]:
with open('data/molecule_data.json', 'r') as data:
    molecule_data = json.load(data)

In [6]:
atoms, coords, multiplicity, charge, basis, sym_sector = molecule_data['B+_STO-3G_SINGLET'].values()

In [7]:
mol.construct_molecule(atoms, coords, charge, multiplicity, basis, taper=False, sym_sector=None)

{'speciesname': 'B1_sto-3g',
 'num_qubits': 10,
 'hamiltonian': {'IIIIIIIIII': (-15.807523675303841+0j),
  'ZIIIIIIIII': (3.977170261095485+0j),
  'YZYIIIIIII': (0.04778786444566182+0j),
  'XZXIIIIIII': (0.04778786444566182+0j),
  'IZIIIIIIII': (3.9771702610954853+0j),
  'IYZYIIIIII': (0.047787864445661764+0j),
  'IXZXIIIIII': (0.047787864445661764+0j),
  'IIZIIIIIII': (0.19716678433985307+0j),
  'IIIZIIIIII': (0.19716678433985307+0j),
  'IIIIZIIIII': (0.050127446443744325+0j),
  'IIIIIZIIII': (0.050127446443744304+0j),
  'IIIIIIZIII': (0.05012744644374435+0j),
  'IIIIIIIZII': (0.05012744644374444+0j),
  'IIIIIIIIZI': (0.05012744644374431+0j),
  'IIIIIIIIIZ': (0.050127446443744325+0j),
  'ZZIIIIIIII': (0.7243233991555813+0j),
  'YIYIIIIIII': (0.08107580474131824+0j),
  'XIXIIIIIII': (0.08107580474131824+0j),
  'ZYZYIIIIII': (0.08107580474131824+0j),
  'ZXZXIIIIII': (0.08107580474131824+0j),
  'YXXYIIIIII': (0.014462511517744511+0j),
  'YYXXIIIIII': (-0.014462511517744511+0j),
  'XXYYII